In [1]:
# Importamos librerías necesarias

import requests
import pandas as pd
import psycopg2
from datetime import datetime

In [2]:
# URL base de la API de CoinCap
base_url = "https://api.coincap.io/v2"


In [3]:
# Endpoint para obtener datos de activos
endpoint = "/assets"

In [4]:
# Encabezados de la solicitud con la clave API
headers = {
    "Accept-Encoding": "gzip, deflate"}

In [5]:
# URL completa para la solicitud
url = base_url + endpoint

In [6]:
# Realizar la solicitud GET a la API
response = requests.get(url, headers=headers)

In [7]:
# Verificar el código de estado de la respuesta
if response.status_code == 200:
    # La solicitud fue exitosa
    data = response.json()
    # Imprimir los datos de los activos (solo imprime los primeros 5 en este ejemplo)
    for asset in data['data'][:5]:
        print(asset)
else:
    # La solicitud falló
    print("Error al realizar la solicitud:", response.status_code)


{'id': 'bitcoin', 'rank': '1', 'symbol': 'BTC', 'name': 'Bitcoin', 'supply': '19658368.0000000000000000', 'maxSupply': '21000000.0000000000000000', 'marketCapUsd': '1379541138576.5233779148320896', 'volumeUsd24Hr': '5248317047.9717464994862041', 'priceUsd': '70175.7713853216796997', 'changePercent24Hr': '0.1435807209903933', 'vwap24Hr': '69945.2007363079532577', 'explorer': 'https://blockchain.info/'}
{'id': 'ethereum', 'rank': '2', 'symbol': 'ETH', 'name': 'Ethereum', 'supply': '120077426.1898597300000000', 'maxSupply': None, 'marketCapUsd': '426013510808.7970615577309671', 'volumeUsd24Hr': '4079956285.5823621395890759', 'priceUsd': '3547.8234696270742473', 'changePercent24Hr': '0.1207086690090205', 'vwap24Hr': '3514.8806637212792651', 'explorer': 'https://etherscan.io/'}
{'id': 'tether', 'rank': '3', 'symbol': 'USDT', 'name': 'Tether', 'supply': '103618183617.7160800000000000', 'maxSupply': None, 'marketCapUsd': '103738958254.0555888473491174', 'volumeUsd24Hr': '15974574796.261312336

In [8]:
# Convertir el JSON a un DataFrame de pandas
df = pd.DataFrame(data['data'])

In [9]:
df

,id,rank,symbol,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,vwap24Hr,explorer
0,bitcoin,1,BTC,Bitcoin,19658368.0000000000000000,21000000.0000000000000000,1379541138576.5233779148320896,5248317047.9717464994862041,70175.7713853216796997,0.1435807209903933,69945.2007363079532577,https://blockchain.info/
1,ethereum,2,ETH,Ethereum,120077426.1898597300000000,None,426013510808.7970615577309671,4079956285.5823621395890759,3547.8234696270742473,0.1207086690090205,3514.8806637212792651,https://etherscan.io/
2,tether,3,USDT,Tether,103618183617.7160800000000000,None,103738958254.0555888473491174,15974574796.2613123361090223,1.0011655737643992,0.0326977855069566,1.0014844079557017,https://www.omniexplorer.info/asset/31
3,binance-coin,4,BNB,BNB,166801148.0000000000000000,166801148.0000000000000000,101238180858.5158090041272288,555217980.0641864618682240,606.9393530703746056,-1.4364626255561608,608.9157890468274129,https://etherscan.io/token/0xB8c77482e45F1F44d...
4,solana,5,SOL,Solana,443884942.2473194000000000,None,87353678929.0497069160607483,1258173072.4917614961566288,196.7935170019325692,4.8220294073198071,193.5879126231700479,https://explorer.solana.com/
...,...,...,...,...,...,...,...,...,...,...,...,...
95,rocket-pool,96,RPL,Rocket Pool,20217174.1801535400000000,None,605048395.6531172134550354,294203.3908528791491162,29.9274463513833244,-3.6048651225215583,30.1155198451119859,https://etherscan.io/token/0xb4efd85c19999d842...
96,frax-share,97,FXS,Frax Share,78569938.8041846500000000,None,595981363.0659362190402028,447186.5682336542340114,7.5853611716723666,-2.6152933824479894,7.6826618563904313,https://etherscan.io/token/0x3432b6a60d23ca0df...
97,holo,98,HOT,Holo,173116448035.6735200000000000,None,593252364.2746532778963693,3793795.6217546368605398,0.0034268977385234,-0.4761227819401331,0.0033822533106279,https://etherscan.io/token/0x6c6ee5e31d828de24...
98,livepeer,99,LPT,Livepeer,31361484.2529945700000000,None,585102428.7505056161266189,10108665.2261084517990523,18.6567199444534186,-1.7871871043587951,18.7921892215340793,https://explorer.livepeer.org/


In [10]:
# Eliminar columnas innecesarias
df.drop(['id','maxSupply', 'vwap24Hr', 'explorer'], axis=1, inplace=True)

In [11]:
df

,rank,symbol,name,supply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr
0,1,BTC,Bitcoin,19658368.0000000000000000,1379541138576.5233779148320896,5248317047.9717464994862041,70175.7713853216796997,0.1435807209903933
1,2,ETH,Ethereum,120077426.1898597300000000,426013510808.7970615577309671,4079956285.5823621395890759,3547.8234696270742473,0.1207086690090205
2,3,USDT,Tether,103618183617.7160800000000000,103738958254.0555888473491174,15974574796.2613123361090223,1.0011655737643992,0.0326977855069566
3,4,BNB,BNB,166801148.0000000000000000,101238180858.5158090041272288,555217980.0641864618682240,606.9393530703746056,-1.4364626255561608
4,5,SOL,Solana,443884942.2473194000000000,87353678929.0497069160607483,1258173072.4917614961566288,196.7935170019325692,4.8220294073198071
...,...,...,...,...,...,...,...,...
95,96,RPL,Rocket Pool,20217174.1801535400000000,605048395.6531172134550354,294203.3908528791491162,29.9274463513833244,-3.6048651225215583
96,97,FXS,Frax Share,78569938.8041846500000000,595981363.0659362190402028,447186.5682336542340114,7.5853611716723666,-2.6152933824479894
97,98,HOT,Holo,173116448035.6735200000000000,593252364.2746532778963693,3793795.6217546368605398,0.0034268977385234,-0.4761227819401331
98,99,LPT,Livepeer,31361484.2529945700000000,585102428.7505056161266189,10108665.2261084517990523,18.6567199444534186,-1.7871871043587951


In [12]:
# Elegimos la columna symbol como ID. Reordenamos y renombramos
symbol_column = df.pop('symbol')  
df.insert(0, 'id', symbol_column)

In [13]:
df

,id,rank,name,supply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr
0,BTC,1,Bitcoin,19658368.0000000000000000,1379541138576.5233779148320896,5248317047.9717464994862041,70175.7713853216796997,0.1435807209903933
1,ETH,2,Ethereum,120077426.1898597300000000,426013510808.7970615577309671,4079956285.5823621395890759,3547.8234696270742473,0.1207086690090205
2,USDT,3,Tether,103618183617.7160800000000000,103738958254.0555888473491174,15974574796.2613123361090223,1.0011655737643992,0.0326977855069566
3,BNB,4,BNB,166801148.0000000000000000,101238180858.5158090041272288,555217980.0641864618682240,606.9393530703746056,-1.4364626255561608
4,SOL,5,Solana,443884942.2473194000000000,87353678929.0497069160607483,1258173072.4917614961566288,196.7935170019325692,4.8220294073198071
...,...,...,...,...,...,...,...,...
95,RPL,96,Rocket Pool,20217174.1801535400000000,605048395.6531172134550354,294203.3908528791491162,29.9274463513833244,-3.6048651225215583
96,FXS,97,Frax Share,78569938.8041846500000000,595981363.0659362190402028,447186.5682336542340114,7.5853611716723666,-2.6152933824479894
97,HOT,98,Holo,173116448035.6735200000000000,593252364.2746532778963693,3793795.6217546368605398,0.0034268977385234,-0.4761227819401331
98,LPT,99,Livepeer,31361484.2529945700000000,585102428.7505056161266189,10108665.2261084517990523,18.6567199444534186,-1.7871871043587951


In [14]:
#Agregamos columna fecha de carga
fecha_hoy = datetime.now().timestamp()
df['fecha_hoy'] = pd.to_datetime(fecha_hoy, unit='s')

In [15]:
df

,id,rank,name,supply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,fecha_hoy
0,BTC,1,Bitcoin,19658368.0000000000000000,1379541138576.5233779148320896,5248317047.9717464994862041,70175.7713853216796997,0.1435807209903933,2024-03-30 15:18:24.783603968
1,ETH,2,Ethereum,120077426.1898597300000000,426013510808.7970615577309671,4079956285.5823621395890759,3547.8234696270742473,0.1207086690090205,2024-03-30 15:18:24.783603968
2,USDT,3,Tether,103618183617.7160800000000000,103738958254.0555888473491174,15974574796.2613123361090223,1.0011655737643992,0.0326977855069566,2024-03-30 15:18:24.783603968
3,BNB,4,BNB,166801148.0000000000000000,101238180858.5158090041272288,555217980.0641864618682240,606.9393530703746056,-1.4364626255561608,2024-03-30 15:18:24.783603968
4,SOL,5,Solana,443884942.2473194000000000,87353678929.0497069160607483,1258173072.4917614961566288,196.7935170019325692,4.8220294073198071,2024-03-30 15:18:24.783603968
...,...,...,...,...,...,...,...,...,...
95,RPL,96,Rocket Pool,20217174.1801535400000000,605048395.6531172134550354,294203.3908528791491162,29.9274463513833244,-3.6048651225215583,2024-03-30 15:18:24.783603968
96,FXS,97,Frax Share,78569938.8041846500000000,595981363.0659362190402028,447186.5682336542340114,7.5853611716723666,-2.6152933824479894,2024-03-30 15:18:24.783603968
97,HOT,98,Holo,173116448035.6735200000000000,593252364.2746532778963693,3793795.6217546368605398,0.0034268977385234,-0.4761227819401331,2024-03-30 15:18:24.783603968
98,LPT,99,Livepeer,31361484.2529945700000000,585102428.7505056161266189,10108665.2261084517990523,18.6567199444534186,-1.7871871043587951,2024-03-30 15:18:24.783603968


In [16]:
# Creamos la conexión a Redsshift
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="matiaspereyra_coderhouse"
with open("C:/Users/machi/Desktop/Cursos/Data Engineer/Coder/pwdredshift.txt",'r') as f:
    pwd= f.read()
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [17]:
#Crear cursor
cursor = conn.cursor()

In [18]:
#Crear la tabla
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS matiaspereyra_coderhouse.crypto
        (
	    id VARCHAR(50) primary key  
	    ,rank INTEGER
	    ,name VARCHAR(255)  
	    ,supply DECIMAL
	    ,marketCapUsd DECIMAL
	    ,volumeUsd24Hr DECIMAL  
	    ,priceUsd DECIMAL 
	    ,changePercent24Hr DECIMAL   
	    ,fecha_hoy TIMESTAMP   
	    )
    """)
    conn.commit()

In [19]:
# Eliminar los registros existentes
cursor.execute("DELETE FROM crypto")
conn.commit()

In [20]:
# Insertamos registros
for index, row in df.iterrows():
    cursor.execute("INSERT INTO crypto (id, rank, name, supply, marketCapUsd, volumeUsd24Hr, priceUsd, changePercent24Hr, fecha_hoy) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                   (row['id'], row['rank'], row['name'], row['supply'], row['marketCapUsd'], row['volumeUsd24Hr'], row['priceUsd'], row['changePercent24Hr'], row['fecha_hoy']))

# Confirmar la inserción de los registros
conn.commit()



In [21]:
# Consulta
query = "SELECT * FROM crypto"

In [22]:
# Leer la tabla en un DataFrame
df_result = pd.read_sql_query(query, conn)

C:\Users\machi\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [23]:
# Cerrar el cursor y la conexión
cursor.close()
conn.close()

In [24]:
# Mostrar el DataFrame resultante
print(df_result)

      id  rank         name        supply  marketcapusd  volumeusd24hr  \
0    BTC     1      Bitcoin  1.965837e+07  1.379541e+12   5.248317e+09   
1    ETH     2     Ethereum  1.200774e+08  4.260135e+11   4.079956e+09   
2   USDT     3       Tether  1.036182e+11  1.037390e+11   1.597457e+10   
3    BNB     4          BNB  1.668011e+08  1.012382e+11   5.552180e+08   
4    SOL     5       Solana  4.438849e+08  8.735368e+10   1.258173e+09   
..   ...   ...          ...           ...           ...            ...   
95   RPL    96  Rocket Pool  2.021717e+07  6.050484e+08   2.942030e+05   
96   FXS    97   Frax Share  7.856994e+07  5.959814e+08   4.471870e+05   
97   HOT    98         Holo  1.731164e+11  5.932524e+08   3.793796e+06   
98   LPT    99     Livepeer  3.136148e+07  5.851024e+08   1.010866e+07   
99    SC   100      Siacoin  5.672512e+10  5.785573e+08   5.020830e+06   

    priceusd  changepercent24hr                  fecha_hoy  
0    70176.0                0.0 2024-03-30 15:18:2